In [94]:
with open('original/vocab.txt', 'r') as f:
    subwords = f.readlines()
    subwords = [subword.strip() for subword in subwords]

In [95]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
        pretrained_model_name_or_path='original/vocab.txt',
        do_lower_case=False,
        unk_token="<unk>",
        sep_token="</s>",
        pad_token="<pad>",
        cls_token="<s>",
        mask_token="<mask>"
    )

In [96]:
result = tokenizer.tokenize("민주주의의 첢첢")[0]
if '<unk>' in result:
    print("ummm")
    print(result)

ummm
['민주주의', '##의', '<unk>']


In [97]:
josa_list = ['이', '가', '을', '를', '은', '는', '에', '에서', '으로', '로', '의' ,'에게' ,'한테' ,'께', '께서',
             '에게서', '한테서', '과', '와', '하고', '이랑', '랑', '도' ,'만', '부터', '까지' ,'이나' ,'마다', '같이',
             '처럼', '보다', '밖에', '대로', '이서' ,'서', '에다가' ,'만큼', '더러', '이라고', '라고', '으로서', '로서',
             '으로써', '로써', '으로서', '로서', '이자', '자', '이며', '며', '말고', '마저', '조차', '뿐', '따라', '치고', '이란', '란',
             '이든지', '든지', '이나마', '나마', '이라도', '라도', '이야말로', '야말로', '은커녕', '는커녕', '인들']

josa_list = sorted(josa_list, key=lambda x : len(x), reverse=True)
print(josa_list)

['이야말로', '에게서', '한테서', '에다가', '이라고', '으로서', '으로써', '으로서', '이든지', '이나마', '이라도', '야말로', '은커녕', '는커녕', '에서', '으로', '에게', '한테', '께서', '하고', '이랑', '부터', '까지', '이나', '마다', '같이', '처럼', '보다', '밖에', '대로', '이서', '만큼', '더러', '라고', '로서', '로써', '로서', '이자', '이며', '말고', '마저', '조차', '따라', '치고', '이란', '든지', '나마', '라도', '인들', '이', '가', '을', '를', '은', '는', '에', '로', '의', '께', '과', '와', '랑', '도', '만', '서', '자', '며', '뿐', '란']


In [98]:
def is_vocab(i, check_subword, subwords):
    if check_subword in subwords[:i]:
        return True
    else:
        result = tokenizer.tokenize(check_subword)[0]
        if '<unk>' in result:
            return False
        else:
            return True

def is_vocab2(i, check_subword, subwords):
    if check_subword in subwords[:i]:
        return True
    return False

In [99]:
is_vocab(2,'에',['아','에','아에'])

True

In [100]:
'나는요'[:len('나는요')-len('요')]

'나는'

In [102]:
max_flag = len(josa_list)

new_subwords = []
cnt = 0
for i, subword in enumerate(subwords):
    flag = 0
    if '##' not in subword: # ##이 subword에 없는 경우
        if len(subword) > 2: #세 글자 이상의 단어에 대해서만 알고리즘 적용함 (부자에서 자도 쪼개버리는 경우가 있어서)
            for josa in josa_list:
                if subword.endswith(josa):
                    if len(subword) > len(josa): #조사를 찾지 않기 위함
                        check_subword = subword[:len(subword)-len(josa)] #조사 제외한 스트링
                        #아래 if문
                        #조사가 vocab에 없어 (조사는 i 위에 있을 필요없음 예를 들어 그나마는 (그나)(마)로 구성됨 (그)(나마)가 아니므로 나마가 위에 안 올 수 있음 
                        #하지만 이건 내가 유도한 바가 아님 그래서 해당 조사가 있으면 vocab 에서 찾아보고 없으면 unused token에서 바꿔주면 됨
                        if is_vocab(i, check_subword, subwords) and is_vocab(i, '##'+josa, subwords):
                            #print(subword)
                            cnt += 1
                        else: ## 조사를 뗀 토큰 혹은 ##조사가 vocab에 없으면 그냥 분리하지말고 저장 이건 조사가 합쳐져서 생긴 것이 아니기 때문
                            new_subwords.append(subword)
                    elif len(subword) == len(josa): ##이건 조사로 쓰인게 아님 조사는 ##으로 시작하지만 우리가 저장한 리스트는 ##이 포함되어 있지 않음
                        new_subwords.append(subword)
                    break
                else: #조사로 끝나지 않으면 flag +1
                    flag += 1
            if flag == max_flag: #조사로 끝나지 않는 단어는 저장
                new_subwords.append(subword)
        else: ## len(subword) <= 2
            new_subwords.append(subword)
            
    elif '##' in subword: # ##이 subword에 있는 경우도 저장
        for jasa in josa_list:
            if subword.endswith(josa):
                if len(subword)-2 > len(josa):
                    check_subword = subword[:len(subword)-len(josa)]
                    if is_vocab2(i, check_subword, subwords) and is_vocab2(i, '##'+josa, subwords):
                        #print(subword)
                        cnt += 1
                    else:
                        new_subwords.append(subword)
                elif subword == '##'+josa:
                    new_subwords.append(subword)
                break
            else: #조사로 끝나지 않으면 flag +1
                flag += 1
        if flag == max_flag: #조사로 끝나지 않는 단어는 저장
            new_subwords.append(subword)

print(len(new_subwords))
print(cnt)

48270
1730


In [103]:
'으'.endswith('으')

True

In [104]:
sub_set = set(subwords)
new_set = set(new_subwords)
print(len(sub_set-new_set))

print(len(subwords), len(sub_set), len(new_subwords), len(new_set))

hash = {}
for i, sw in enumerate(new_subwords):
    if sw not in hash:
        hash[sw] = 1
    else:
        hash[sw] += 1

for k,v in hash.items():
    if v > 1:
        print(k)

1730
50000 50000 48270 48270


In [109]:
cha = list(sub_set - new_set)
#print(cha)

In [106]:
print(cha[:50])

['제국주의', '원자로', '레이나', '집단주의', '걸으며', '불러서', '깽깽이', '어서라도', '시켰으며', '떠돌이', '닌텐도', '객관적으로', '초현실주의', '유발자', '딸랑딸랑', '유니클로', '할려는', '제주도', '출연자', '유단자', '시위자', '곡괭이', '전라도', '불러다가', '야요이', '자유민주주의', '훔쳐서', '생산자', '막아서', '들썩이', '계승자', '한데도', '돌연변이', '저래도', '떠벌이', '해상도', '따라가', '살인자', '자살자', '라면서', '이력서', '주권자', '더니만', '여기자', '아카이', '위키백과', '지배자', '역사가', '눈덩이', '발주자']


In [107]:
print(new_subwords[-30:-20])

['복덕방', '레반', '뭇매', '참살', '서경덕', '볼지', '영력', '피날레', '##사쿠', '인터폴']


In [108]:
vocab_file = 'v4/vocab.txt'
with open(vocab_file,'w',encoding='utf-8') as f:
    for w in new_subwords:  
        f.write(w+'\n')

In [7]:
import importlib
module = importlib.import_module(f".bert", "transformers.models")

In [10]:
def getattribute_from_module(module, attr):
    if attr is None:
        print('1')
        return None
    if isinstance(attr, tuple):
        print('2')
        return tuple(getattribute_from_module(module, a) for a in attr)
    if hasattr(module, attr):
        print('3')
        return getattr(module, attr)
    # Some of the mappings have entries model_type -> object of another model type. In that case we try to grab the
    # object at the top level.
    transformers_module = importlib.import_module("transformers")
    return getattribute_from_module(transformers_module, attr)

In [11]:
print(getattribute_from_module(module, 'BertForMaskedLM'))

3
<class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>
